In [1]:
import os
import pydicom
import matplotlib.pyplot as plt
import pandas as pd
import shutil
import numpy as np
from scipy.optimize import curve_fit
from tqdm import tqdm

In [2]:
import SimpleITK as sitk
import pyvista as pv

In [21]:
from typing import Tuple, List
root = "/Users/yaesu539/Library/CloudStorage/OneDrive-Uppsalauniversitet/PhD/DATA/MRI/20230425_FakeIntestines_DLS/Fake_Intestine_A1_P1/DICOM/T2"


def read_supervolume(root) -> Tuple[np.ndarray, List[float]]:
    """Read a folder with one folder per echo time. Assumes that the folder names are the echo times

    Args:
        root (str): root dir where all the echo times live

    Returns:
        Tuple[np.ndarray, List[float]]: 
            Volume with shape (E, X, Y, Z) where E is the echo time.
            List with the sorted echo times (names of the folders).
    """
    volumes = []
    echo_times = []
    max_shape = None
    for dir in sorted([x for x in os.listdir(root) if os.path.isdir(os.path.join(root, x))], key=lambda x: float(x)):
        echo_times.append(float(dir))
        dir = os.path.join(root, dir)
        print(dir)
        reader = sitk.ImageSeriesReader()
        dicom_names = reader.GetGDCMSeriesFileNames(dir)
        reader.SetFileNames(dicom_names)
        image = reader.Execute()
        volume = sitk.GetArrayFromImage(image)
        # Update max_shape if necessary
        if max_shape is None:
            max_shape = volume.shape
        else:
            max_shape = np.maximum(max_shape, volume.shape)
        
        volumes.append(volume)
    
    # Pad each volume with zeros to match the maximum shape
    padded_volumes = []
    for volume in volumes:
        pad_width = [(0, max_shape[i] - volume.shape[i]) for i in range(3)]
        padded_volume = np.pad(volume, pad_width=pad_width, mode='constant', constant_values=0)
        padded_volumes.append(padded_volume)
    
    volumes = np.stack(padded_volumes)
    #     volumes.append(sitk.GetArrayFromImage(image))
    # volumes = np.stack(volumes)
    echo_times = [et/1000 for et in echo_times]
    return volumes, echo_times

super_volume, echo_times = read_supervolume(root)

/Users/yaesu539/Library/CloudStorage/OneDrive-Uppsalauniversitet/PhD/DATA/MRI/20230425_FakeIntestines_DLS/Fake_Intestine_A1_P1/DICOM/T2/8.0
/Users/yaesu539/Library/CloudStorage/OneDrive-Uppsalauniversitet/PhD/DATA/MRI/20230425_FakeIntestines_DLS/Fake_Intestine_A1_P1/DICOM/T2/16.0
/Users/yaesu539/Library/CloudStorage/OneDrive-Uppsalauniversitet/PhD/DATA/MRI/20230425_FakeIntestines_DLS/Fake_Intestine_A1_P1/DICOM/T2/24.0
/Users/yaesu539/Library/CloudStorage/OneDrive-Uppsalauniversitet/PhD/DATA/MRI/20230425_FakeIntestines_DLS/Fake_Intestine_A1_P1/DICOM/T2/32.0
/Users/yaesu539/Library/CloudStorage/OneDrive-Uppsalauniversitet/PhD/DATA/MRI/20230425_FakeIntestines_DLS/Fake_Intestine_A1_P1/DICOM/T2/40.0
/Users/yaesu539/Library/CloudStorage/OneDrive-Uppsalauniversitet/PhD/DATA/MRI/20230425_FakeIntestines_DLS/Fake_Intestine_A1_P1/DICOM/T2/48.0
/Users/yaesu539/Library/CloudStorage/OneDrive-Uppsalauniversitet/PhD/DATA/MRI/20230425_FakeIntestines_DLS/Fake_Intestine_A1_P1/DICOM/T2/56.0
/Users/yaesu53

In [26]:
def fit_r(echo_time, mean_intensity):
    # plt.scatter(echo_time, mean_intensity)
    # plt.show()
    def func(x, I0, rs):
        return I0 * np.exp(-(rs)*x)
    popt, pcov = curve_fit(func, echo_time, mean_intensity, )
    return popt

def fit_params_over_echotime(super_volume):
    """For each voxel, fit the params over the echo_time

    Args:
        super_volume (ndarray): 4D ndarray with shape (E, X, Y, Z)

    Returns:
        ndarray: The `rs` parameter fitter to each voxel with shape (X, Y, Z)
    """
    s = super_volume.shape
    fitted_volume = np.zeros([s[1], s[2], s[3]])
    pbar = tqdm(desc='fitting params', total=s[1]*s[2]*s[3], mininterval=0.5, unit="voxel")
    for x in range(s[1]):
        for y in range(s[2]):
            for z in range(s[3]):
                try:
                    I0, rs = fit_r(echo_times, super_volume[:, x, y, z])
                    fitted_volume[x, y, z] = rs
                    # input()
                except RuntimeError as e:
                    print(x, y, z, e)
                    fitted_volume[x, y, z] = 0
                pbar.update()
    return fitted_volume

# select a smaller volume to make the fitting faster
s = super_volume.shape
xlim = [1*s[1]//10, 10*s[1]//10]
ylim = [4*s[2]//20, 9*s[2]//20]
zlim = [1*s[3]//10, 10*s[3]//10]
super_volume_crop = super_volume[:, xlim[0]:xlim[1], ylim[0]:ylim[1], zlim[0]:zlim[1]]
#super_volume_crop = super_volume[:, xlim[0]:xlim[1], ylim[0]:ylim[1], zlim[0]:zlim[1]]
#super_volume_crop = super_volume
# fit the params
fitted_volume = fit_params_over_echotime(super_volume_crop)


fitting params: 100%|██████████| 1505100/1505100 [04:42<00:00, 5322.60voxel/s]


In [27]:
print(super_volume.shape)

(26, 192, 300, 128)


In [28]:
# save volume for fast loading
save_path = os.path.join(root,"fitted_volume_4-4p5_ms.npz")
np.savez(save_path, rs=fitted_volume)

In [29]:
# load volume
f = np.load(save_path)
fitted_volume = f['rs']

Run plot_mri_fer.py